In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [2]:
import json
import pandas as pd
import numpy as np
import os
import sys
import json
import matplotlib.pyplot as plt
import matplotlib.patches as pch
import cv2
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np
import math, random
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
from collections import defaultdict
from tqdm import tqdm
import tensorflow as tf
import torch
import torchvision
import torch.utils.data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np
import math, random
import pandas as pd
import pathlib
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import copy
import time 
from collections import defaultdict
from tqdm import tqdm
import os
import json


# a=json.load('/kaggle/input/benetech-making-graphs-accessible/train/annotations/cc68f19b708c.json')

In [3]:
class CFG:
    isOneHot = False
    label_map = {'dot': 0, 'horizontal_bar' : 1, 'vertical_bar': 2, 'line': 3, 'scatter': 4}
    num_classes = 5
    batchSize = 32
    fold_train = 0

In [4]:
test_images=[]
for i in os.listdir('/kaggle/input/benetech-making-graphs-accessible/test/images'):
    test_images.append(i)

In [5]:
test_images

['000b92c3b098.jpg',
 '01b45b831589.jpg',
 '00f5404753cf.jpg',
 '00dcf883a459.jpg',
 '007a18eb4e09.jpg']

In [6]:
# source,chart-type plot_bb(height,width,x0,y0), text(id,polygon(x0-x3,y0-y3),text,role)

In [7]:
def convert_annot2df(path):
    res = []
    col_name = ['name', 'source', 'chart-type', 'plot-bb', 'text', 'x_axis', 'y_axis', 'data_series_x', 'data_series_y']
    for i in tqdm(os.listdir(path)):
        tmp_name = i[:-5]
        tmp_data = [tmp_name]
        tmp_json_file_path = os.path.join(path, i)
        with open(tmp_json_file_path, 'r') as tmp_file:
            tmp_json_data = json.load(tmp_file)
        
        tmp_data.append(tmp_json_data['source']) # generated / extracted
        tmp_data.append(tmp_json_data['chart-type']) # dot, horizontal_bar, vertical_bar, line, scatter.
        tmp_data.append(','.join(map(str, [tmp_json_data['plot-bb']['height'], tmp_json_data['plot-bb']['width'], 
                                           tmp_json_data['plot-bb']['x0'], tmp_json_data['plot-bb']['y0']])))
        
        # text
        tmp_data.append(tmp_json_data['text'])
        # axes
        tmp_data.append(tmp_json_data['axes']['x-axis'])
        tmp_data.append(tmp_json_data['axes']['y-axis'])
        # data_series_x
        tmp_data_series_x = []
        # data_series_y
        tmp_data_series_y = []
        ground_truth=[]
        for ds in tmp_json_data['data-series']:
            tmp_data_series_x.append(str(ds['x']))
            tmp_data_series_y.append(str(ds['y']))
        tmp_data.append(';'.join(tmp_data_series_x))
        tmp_data.append(';'.join(tmp_data_series_y))
        
        res.append(tmp_data)
    return pd.DataFrame(res, columns=col_name)


In [8]:
annotation_df = convert_annot2df('/kaggle/input/benetech-making-graphs-accessible/train/annotations')

100%|██████████| 60578/60578 [05:39<00:00, 178.36it/s]


In [9]:
annotation_df.head(1)

,name,source,chart-type,plot-bb,text,x_axis,y_axis,data_series_x,data_series_y
0,cc68f19b708c,extracted,line,"236,426,63,15","[{'id': 0, 'polygon': {'x0': 11, 'x1': 23, 'x2...","{'ticks': [{'id': 11, 'tick_pt': {'x': 116, 'y...","{'ticks': [{'id': 1, 'tick_pt': {'x': 63, 'y':...",10%;20%;30%;40%;50%;60%;70%,1.5481051373212678;11.520004347083574;14.05428...


In [10]:
annotation_df['ground_truth'] = '<|BOS|><'+ annotation_df['chart-type'] + '><x_start>' + annotation_df['data_series_x']+ '<x_end><y_start>'+ annotation_df['data_series_y'] + '<y_end>'

In [11]:
annotation_df.head(1)

,name,source,chart-type,plot-bb,text,x_axis,y_axis,data_series_x,data_series_y,ground_truth
0,cc68f19b708c,extracted,line,"236,426,63,15","[{'id': 0, 'polygon': {'x0': 11, 'x1': 23, 'x2...","{'ticks': [{'id': 11, 'tick_pt': {'x': 116, 'y...","{'ticks': [{'id': 1, 'tick_pt': {'x': 63, 'y':...",10%;20%;30%;40%;50%;60%;70%,1.5481051373212678;11.520004347083574;14.05428...,<|BOS|><line><x_start>10%;20%;30%;40%;50%;60%;...


In [12]:
# annotation_df['len_x']=annotation_df['data_series_x'].apply(lambda x: len(x.split(';')))
# annotation_df['len_y']=annotation_df['data_series_y'].apply(lambda x: len(x.split(';')))

In [13]:
annotation_df['fname']=annotation_df['name'].apply(lambda x: '/kaggle/input/benetech-making-graphs-accessible/train/images/'+x+'.jpg')

In [14]:
type_list=annotation_df['chart-type'].value_counts().index.to_list()
chart_category={}
for i in range(0,5):
    chart_category[type_list[i]]=i
def chart_type_category(label):
    index= chart_category[label]
    return index

In [15]:
annotation_df['ct_index']=annotation_df['chart-type'].apply(lambda x: chart_type_category(x))

In [16]:
# from tensorflow.keras.preprocessing.image import load_img
# sample=load_img(annotation_df['fname'][10])
# sample
# # s=cv2.resize(sample,(300,300))

In [17]:
# from tensorflow.keras.preprocessing.image import img_to_array
# sample_array.shape

In [18]:
hFlip=transforms.RandomHorizontalFlip(p=0.25)
vFlip=transforms.RandomVerticalFlip(p=0.25)
transforms_train = transforms.Compose([
                                       transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transforms_val = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transforms_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [19]:
class ImageCharts(torch.utils.data.Dataset):
    def __init__(self, df, transforms  = None): ### phase = train/test
        self.df = df
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        name_img = self.df.iloc[idx]['fname']
        img = cv2.imread(name_img)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(500,300))
        img = img.astype(np.float32)/255.0

        label = np.array(self.df.iloc[idx]['ct_index'])

        if(self.transforms is not None):
            img = self.transforms(img)
        return img, label

In [20]:
def cv_split(Xtrain, ytrain, n_folds, seed):
    kfold = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = seed)
    for num, (train_index, val_index) in enumerate(kfold.split(Xtrain, ytrain)):
        Xtrain.loc[val_index, 'fold'] = int(num)
    Xtrain['fold'] = Xtrain['fold'].astype(int)
    return Xtrain

annotation_df = cv_split(annotation_df,annotation_df['ct_index'], 5, 42)

In [21]:
train_ds = ImageCharts(annotation_df[annotation_df.fold != CFG.fold_train].reset_index(), transforms=transforms_train)
val_ds = ImageCharts(annotation_df[annotation_df.fold == CFG.fold_train].reset_index(), transforms=transforms_val)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size= CFG.batchSize, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size= CFG.batchSize, shuffle=True)

In [22]:
model = torchvision.models.resnet50(pretrained=False)


num_features = model.fc.in_features
model.fc = nn.Linear(num_features, CFG.num_classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = model.to(device)
next(myModel.parameters()).device

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


device(type='cuda', index=0)

In [23]:
def training(model, train_dl, val_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

    # Repeat for each epoch
    best_acc = -1
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

    # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            if (i + 1) % 50 == 0:    # print every 10 mini-batches
                print('Epoch [{}/{}], Step [{}/{}], Loss : {:.4f}'.format(epoch + 1, num_epochs, i + 1, len(train_dl), running_loss/(i + 1)))
    
        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction/total_prediction
        print(f'Epoch: {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')

        gt = []
        pred = []

        with torch.no_grad():
            correct = 0
            total = 0
            val_loss = 0 
            for idx, data_ in enumerate(val_dl):
                inputs, labels = data_[0].to(device), data_[1].to(device)

                # Normalize the inputs
                inputs_m, inputs_s = inputs.mean(), inputs.std()
                inputs = (inputs - inputs_m) / inputs_s

                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Keep stats for Loss and Accuracy
                val_loss += loss.item()

                # Get the predicted class with the highest score
                _, prediction = torch.max(outputs,1)
                predi = torch.softmax(outputs, dim = -1)
                gt.append(labels)
                pred.append(predi[:,1])
                # Count of predictions that matched the target label
                correct += (prediction == labels).sum().item()
                total += prediction.shape[0]
            print('Accuracy of the network val: {:.4f} %'.format(100 * correct / total))

            final_score = 100 * correct / total


            if (best_acc < final_score):
                best_acc = final_score
                print("Saving best model!")
            torch.save(model.state_dict(), f'Benetech _ResNet50_fold0.pth')

    print('Finished Training')
  
num_epochs=2   # Just for demo, adjust this higher.
training(myModel, train_dl, val_dl, num_epochs)

Epoch [1/2], Step [50/1515], Loss : 1.1217
Epoch [1/2], Step [100/1515], Loss : 0.8408
Epoch [1/2], Step [150/1515], Loss : 0.6875
Epoch [1/2], Step [200/1515], Loss : 0.5999
Epoch [1/2], Step [250/1515], Loss : 0.5344
Epoch [1/2], Step [300/1515], Loss : 0.4856
Epoch [1/2], Step [350/1515], Loss : 0.4436
Epoch [1/2], Step [400/1515], Loss : 0.4153
Epoch [1/2], Step [450/1515], Loss : 0.3922
Epoch [1/2], Step [500/1515], Loss : 0.3717
Epoch [1/2], Step [550/1515], Loss : 0.3528
Epoch [1/2], Step [600/1515], Loss : 0.3335
Epoch [1/2], Step [650/1515], Loss : 0.3166
Epoch [1/2], Step [700/1515], Loss : 0.3040
Epoch [1/2], Step [750/1515], Loss : 0.2901
Epoch [1/2], Step [800/1515], Loss : 0.2778
Epoch [1/2], Step [850/1515], Loss : 0.2669
Epoch [1/2], Step [900/1515], Loss : 0.2558
Epoch [1/2], Step [950/1515], Loss : 0.2460
Epoch [1/2], Step [1000/1515], Loss : 0.2362
Epoch [1/2], Step [1050/1515], Loss : 0.2282
Epoch [1/2], Step [1100/1515], Loss : 0.2206
Epoch [1/2], Step [1150/1515],

In [24]:
# sample_rec_data = sample_data['text'].values[0]
# fig = plt.figure()
# ax = fig.add_subplot(111)
# image = cv2.imread('/kaggle/input/benetech-making-graphs-accessible/train/images/'+sample_data['name'].values[0]+'.jpg')
# plt.imshow(image)

# for i in range(len(sample_rec_data)):
#     rect = pch.Rectangle(xy=(sample_rec_data[i]['polygon']['x0'], sample_rec_data[i]['polygon']['y0']), 
#                            width=sample_rec_data[i]['polygon']['x1'] - sample_rec_data[i]['polygon']['x0'],
#                            height=sample_rec_data[i]['polygon']['y3'] - sample_rec_data[i]['polygon']['y0'],
#                         color='red', fill=False)
#     ax.add_patch(rect)